Applied Capstone Project: Week 3 Assignment Part-1

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

In [4]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [6]:
df_toronto = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])

In [7]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [8]:
df_toronto = df_toronto.dropna()

In [9]:
empty = 'Not assigned'
df_toronto = df_toronto[(df_toronto.PostalCode != empty ) & (df_toronto.Borough != empty) & (df_toronto.Neighborhood != empty)]

In [10]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [15]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df_toronto.groupby(['PostalCode', 'Borough'])
df_toronto2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [16]:
df_toronto2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_toronto2.shape

(102, 3)

Applied Capstone Project: Week 3 Assignment Part-2

In [25]:
!pip install geocoder
import geocoder

In [27]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70949500000006, -79.36398897099997]

In [29]:
postal_codes = df_toronto2['PostalCode']  
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [32]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_toronto2['Latitude'] = df_coords['Latitude']
df_toronto2['Longitude'] = df_coords['Longitude']

In [33]:
df_toronto2.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


Applied Capstone Project: Week 3 Assignment Part-3

In [35]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.3.0               |           py36_0         747 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         846 KB

The following NEW packages will be INSTALLED:

    altair:  3.3.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | #################################

In [36]:
df3 = df_toronto2[ df_toronto2.Borough.str.contains('Toronto') ]

In [37]:
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676531,-79.295425
41,M4K,East Toronto,"Riverdale, The Danforth West",43.683178,-79.355105
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.667965,-79.314667
43,M4M,East Toronto,Studio District,43.660629,-79.334855
44,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
45,M4P,Central Toronto,Davisville North,43.712755,-79.388514
46,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
47,M4S,Central Toronto,Davisville,43.703395,-79.385964
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.690685,-79.382946
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686074,-79.402265


In [65]:

toronto_data = pd.DataFrame(df3)
toronto_data = toronto_data.reset_index().drop('index', axis=1)

toronto_coords = get_latlng('')
toronto_coords

[43.648690000000045, -79.38543999999996]

In [39]:
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [41]:
neighborhood_latitude = df3.loc[41, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df3.loc[41, 'Longitude'] # neighborhood longitude value

neighborhood_name = df3.loc[41, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Riverdale, The Danforth West are 43.68317787500007, -79.35510499999998.


In [42]:
VERSION = '20191208'
CLIENT_ID = 'DZQAVAYYXABUESFPSLUB4SXRCGTVABIGTUHI02SJ3RKY1OL5'
CLIENT_SECRET = 'GQSOI3QBIR0P2LIJABPYZTS2LINOTNZ3RGEKRAPZR1PGT25G'
latitude = neighborhood_latitude
longitude = neighborhood_longitude
radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius, LIMIT)

In [43]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5decbeef542890001bc2b137'},
 'response': {'headerLocation': 'Greektown',
  'headerFullLocation': 'Greektown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.68767787950007,
    'lng': -79.34889400848712},
   'sw': {'lat': 43.67867787050007, 'lng': -79.36131599151284}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05cf964a520f5f520e3',
       'name': "Whistler's Grille and The McNeil Room",
       'location': {'address': '995 Broadview Avenue',
        'crossStreet': 'Mortimer Ave',
        'lat': 43.684082064401245,
        'lng': -79.35678903213018,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.684082064401245,
          'lng': -79.3567

In [44]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Whistler's Grille and The McNeil Room,Restaurant,43.684082,-79.356789
1,Dollarama,Discount Store,43.686197,-79.355989
2,Charles Sauriol Parkette,Park,43.685270,-79.356588
3,TTC Bus #8 Broadview,Bus Line,43.687101,-79.355078
4,Sobeys Todmorden,Grocery Store,43.684723,-79.356461


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [48]:
toronto_venues = getNearbyVenues(df3['Neighborhood'], df3['Latitude'], df3['Longitude'])

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [49]:
toronto_venues.head()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676531,-79.295425,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West",43.683178,-79.355105,Whistler's Grille and The McNeil Room,43.684082,-79.356789,Restaurant


In [50]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",71,71,71,71,71,71
Berczy Park,62,62,62,62,62,62
"Brockton, Exhibition Place, Parkdale Village",67,67,67,67,67,67
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,99,99,99,99,99,99
"Chinatown, Grange Park, Kensington Market",80,80,80,80,80,80
Christie,11,11,11,11,11,11


In [51]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))


There are 224 uniques categories.


In [52]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.030000,0.000000,0.000000,0.010000,0.000000,0.030000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.014085,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,...,0.0000,0.000000,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.014925,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.029851,0.000000,0.014925,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000,0.000000,0.000000,...,0.0000,0.010101,0.000000,0.000000,0.000000,0.000000,0.010101,0.010101,0.010101,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,0.012500,0.000000,0.000000,...,0.0125,0.000000,0.000000,0.000000,0.000000,0.025000,0.000000,0.037500,0.012500,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.012346,0.012346,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346


In [54]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2        Hotel  0.05
3   Steakhouse  0.04
4          Bar  0.03


----Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara----
                  venue  freq
0           Coffee Shop  0.08
1    Italian Restaurant  0.07
2  Gym / Fitness Center  0.04
3                   Bar  0.04
4                   Pub  0.03


----Berczy Park----
         venue  freq
0  Coffee Shop  0.08
1   Restaurant  0.05
2       Bakery  0.03
3         Café  0.03
4  Cheese Shop  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.06
2              Restaurant  0.06
3  Furniture / Home Store  0.06
4                  Bakery  0.06


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.10
1        Hotel  0.04
2          Bar  0.04
3   S

In [55]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [56]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Gastropub,Bakery,Japanese Restaurant,Restaurant,Bar,Burger Joint
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",Coffee Shop,Italian Restaurant,Bar,Gym / Fitness Center,Sandwich Place,Park,Speakeasy,Restaurant,French Restaurant,Bakery
2,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Beer Bar,Café,Bakery,Farmers Market,Breakfast Spot,Cheese Shop,Hotel
3,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Bakery,Furniture / Home Store,Restaurant,Sandwich Place,Italian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Supermarket
4,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Steakhouse,Bar,Café,Hotel,Pizza Place,Restaurant,Sushi Restaurant,Pub,Thai Restaurant
5,"Cabbagetown, St. James Town",Coffee Shop,Chinese Restaurant,Pizza Place,Bakery,Italian Restaurant,Restaurant,Café,Farm,Market,Pub
6,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Fast Food Restaurant,Tea Room,Hotel,Plaza,Sushi Restaurant,Chinese Restaurant,Sandwich Place
7,"Chinatown, Grange Park, Kensington Market",Café,Chinese Restaurant,Bar,Mexican Restaurant,Dumpling Restaurant,Vietnamese Restaurant,Bakery,Coffee Shop,Caribbean Restaurant,Tea Room
8,Christie,Café,Grocery Store,Coffee Shop,Candy Store,Italian Restaurant,Playground,Baby Store,Elementary School,Ethiopian Restaurant,Wings Joint
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Dance Studio,Men's Store,Hotel,Gastropub,Bubble Tea Shop


In [57]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)